In [3]:
import pandas as pd

In [80]:
data_dir = 'data/mimic-iv-1.0/physionet.org/files/mimiciv/1.0/'
core = data_dir+'core/' 
hosp = data_dir+'hosp/'
icu = data_dir+'icu/'

In [81]:
patients = pd.read_csv(core+'patients.csv')
admissions = pd.read_csv(core+'admissions.csv')
transfers = pd.read_csv(core+'transfers.csv')
drgs = pd.read_csv(hosp+'drgcodes.csv')

In [106]:
neonates = patients[patients['anchor_age'] == 0].copy()
print(f"Total neonates: {len(neonates)}")

Total neonates: 60872


,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,marital_status,ethnicity,edregtime,edouttime,hospital_expire_flag
0,14679932,21038362,2139-09-26 14:16:00,2139-09-28 11:30:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0
1,15585972,24941086,2123-10-07 23:56:00,2123-10-12 11:22:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,WHITE,NaN,NaN,0
2,11989120,21965160,2147-01-14 09:00:00,2147-01-17 14:25:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,UNKNOWN,NaN,NaN,0
3,17817079,24709883,2165-12-27 17:33:00,2165-12-31 21:18:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,OTHER,NaN,NaN,0
4,15078341,23272159,2122-08-28 08:48:00,2122-08-30 12:32:00,NaN,ELECTIVE,NaN,HOME,Other,ENGLISH,NaN,BLACK/AFRICAN AMERICAN,NaN,NaN,0


In [98]:
nicu_transfers = transfers[transfers['careunit'] == 'Neonatal Intensive Care Unit (NICU)']
neonates_nicu = pd.merge(neonates, nicu_transfers, on='subject_id', how='inner')
neonates_nicu = neonates_nicu[neonates_nicu['eventtype'] == 'admit']
print(f"Neonates with NICU transfers: {len(neonates_nicu)}")

Neonates with NICU transfers: 7758


In [99]:
neonates_nicu.head()

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod,hadm_id,transfer_id,eventtype,careunit,intime,outtime
0,10005325,F,0,2154,2011 - 2013,NaN,20047480.0,39663534,admit,Neonatal Intensive Care Unit (NICU),2154-12-15 14:23:12,2154-12-15 14:28:49
3,10008101,M,0,2142,2008 - 2010,NaN,29646360.0,33581740,admit,Neonatal Intensive Care Unit (NICU),2142-12-26 21:20:39,2142-12-26 22:47:01
5,10012663,F,0,2171,2011 - 2013,NaN,27340067.0,39415613,admit,Neonatal Intensive Care Unit (NICU),2171-05-18 11:05:04,2171-05-18 11:14:30
9,10014536,F,0,2113,2008 - 2010,NaN,24693529.0,37628162,admit,Neonatal Intensive Care Unit (NICU),2113-12-19 18:19:41,2113-12-19 18:23:09
19,10025785,F,0,2155,2008 - 2010,NaN,27677536.0,36967998,admit,Neonatal Intensive Care Unit (NICU),2155-06-14 18:58:56,2155-06-14 19:00:26


In [100]:
premature_drg_codes = [791, 792]
drg_premature = drgs[drgs['drg_code'].isin(premature_drg_codes)]
premature_neonates = pd.merge(neonates_nicu, drg_premature, on=['subject_id','hadm_id'], how='inner')
print(f"Premature neonates: {len(premature_neonates)}")

Premature neonates: 1958


In [101]:
neonates_mortality = pd.merge(
    premature_neonates,
    admissions[['subject_id','hadm_id','hospital_expire_flag']],
    on=['subject_id','hadm_id'],
    how='left'
)

In [102]:
neonates_mortality['hospital_expire_flag'].value_counts()

hospital_expire_flag
0    1958
Name: count, dtype: int64

In [103]:
neonates_mortality['dod'].isnull().sum()

np.int64(1958)

In [104]:
len(neonates_mortality['dod']) - neonates_mortality['dod'].isnull().sum()

np.int64(0)

In [118]:
neonates_admissions = pd.merge(
    admissions,
    premature_neonates,
    on=['subject_id'],
    how='inner'
)
print(f"Premature admissions: {len(neonates_admissions)}")

Premature admissions: 2019
